# Morbilidad Hospitalaria España 2016-2019
## Creación dataframe para visualización

[Datos INE](https://www.ine.es/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736176778&menu=resultados&secc=1254736195291&idp=1254735573175#!tabs-1254736195291)

In [67]:
import pandas as pd
import datetime
import json
from shapely.geometry import shape
import geopandas as gpd
import geojson

In [2]:
#tenemos 4 dataframes por años desde 2016 a 2019
morbi16 = pd.read_csv('../data/morbilidad/EMH_2016_G01.csv',sep='\t')
morbi17 = pd.read_csv('../data/morbilidad/EMH_2017_G01.csv',sep='\t')
morbi18 = pd.read_csv('../data/morbilidad/EMH_2018_G01.csv',sep='\t')
morbi19 = pd.read_csv('../data/morbilidad/EMH_2019_G01.csv',sep='\t')

In [3]:
morbilidad = pd.concat([morbi16,morbi17,morbi18,morbi19])

In [4]:
morbilidad.sample(5)

,Norden,ProvHosp,Sexo,ProvResi,DiagEntr,FxAlta,DiagPrin,MotivoAlta,EdadAnios,EdadMes,EdadDias,DiasEstancia,Factor
286248,40689194,46,2,46,1,22122017,C50912,1,55,9,20,2,1.00000
889747,5487556,8,2,8,2,30042019,G500,1,82,5,10,1,1.08056
622155,19122929,21,2,21,1,10042016,E210,1,45,9,12,3,1.76245
721581,10050789,8,2,8,1,18032016,F3112,1,53,9,30,40,1.02915
448440,224531,2,1,16,2,22092018,C8522,1,30,3,28,15,1.00000


In [5]:
#el código de las enferemades mentales empeizan por F (hacemos una selección)
morbilidad = morbilidad[morbilidad["DiagPrin"].str.contains('^F')]
morbilidad.DiagPrin = morbilidad.DiagPrin.str.strip()

In [6]:
morbilidad.drop(['Norden','EdadDias','EdadMes'],axis=1,inplace = True)

In [7]:
morbilidad.DiagPrin.unique()[0:20] #extraer F + 2 primero dígitos para tener los tipos de enfermedasdes mentales agrupadas

array(['F0150', 'F0151', 'F0390', 'F0391', 'F04', 'F05', 'F060', 'F061',
       'F062', 'F0630', 'F0631', 'F0632', 'F0633', 'F0634', 'F064',
       'F068', 'F070', 'F0781', 'F0789', 'F079'], dtype=object)

In [8]:
morbilidad["code"] = morbilidad.DiagPrin.str.extract(r'(^F..)')

In [9]:
morbilidad.code.unique()

array(['F01', 'F03', 'F04', 'F05', 'F06', 'F07', 'F09', 'F10', 'F11',
       'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20',
       'F21', 'F22', 'F23', 'F24', 'F25', 'F28', 'F29', 'F30', 'F31',
       'F32', 'F33', 'F34', 'F39', 'F40', 'F41', 'F42', 'F43', 'F44',
       'F45', 'F48', 'F50', 'F51', 'F52', 'F53', 'F55', 'F59', 'F60',
       'F63', 'F64', 'F65', 'F66', 'F68', 'F69', 'F70', 'F71', 'F72',
       'F73', 'F78', 'F79', 'F80', 'F81', 'F82', 'F84', 'F88', 'F89',
       'F90', 'F91', 'F93', 'F94', 'F95', 'F98', 'F99'], dtype=object)

## Añadimos códigos CIE (Capítulos y diagnósticos)

In [10]:
#importamos la tabla de correspondencia entre código de diagnóstico agrupadas y nombre
diagnos = pd.read_csv("../data/diagnos.csv")

In [11]:
diagnos.sample(3) 

,code,diag,cap,title
56,F70,Retraso mental leve,F70-79,Retraso mental
11,F11,Trastornos mentales y de comportamiento debido...,F10-F19,Trastornos mentales y de comportamiento debido...
3,F03,Demencia sin especificar,F00-F09,"Trastornos mentales orgánicos, incluidos los t..."


In [12]:
diagnos.code.unique()

array(['F00', 'F01', 'F02', 'F03', 'F04', 'F05', 'F06', 'F07', 'F08',
       'F09', 'F10', 'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F17',
       'F18', 'F19', 'F20', 'F21', 'F22', 'F23', 'F24', 'F25', 'F28',
       'F29', 'F30', 'F31', 'F32', 'F33', 'F34', 'F39', 'F40', 'F41',
       'F42', 'F43', 'F44', 'F45', 'F48', 'F50', 'F51', 'F52', 'F53',
       'F54', 'F55', 'F59', 'F60', 'F62', 'F63', 'F64', 'F66', 'F68',
       'F69', 'F65', 'F70', 'F71', 'F72', 'F73', 'F78', 'F79', 'F80',
       'F81', 'F82', 'F83', 'F84', 'F88', 'F89', 'F90', 'F91', 'F92',
       'F93', 'F94', 'F95', 'F98', 'F99'], dtype=object)

In [13]:
dicc_diag = {} #crear diccionario con diagnósticos y códigos
for i,r in diagnos.iterrows():
    dicc_diag[r.code] = r.diag

In [14]:
dicc_cap = {} #crear diccionario con diagnósticos y códigos
for i,r in diagnos.iterrows():
    dicc_cap[r.code] = r.cap

In [15]:
morbilidad["cap"] = morbilidad.code.map(dicc_cap) #creamos nueva columnas

In [16]:
morbilidad.sample(3)

,ProvHosp,Sexo,ProvResi,DiagEntr,FxAlta,DiagPrin,MotivoAlta,EdadAnios,DiasEstancia,Factor,code,cap
696743,38,2,38,2,2102018,F1024,1,42,4,1.05877,F10,F10-F19
740859,10,2,10,2,21082019,F209,1,27,39,1.00000,F20,F20-29
685146,50,1,50,2,28032017,F0390,3,88,5,1.00000,F03,F00-F09


In [17]:
discapacidad = morbilidad[morbilidad["cap"] == 'F70-79'].index #dropeamos el tipo de retraso mental
morbilidad.drop(discapacidad,inplace = True)

## Creamos fecha de ingreso

In [18]:
#creamos fecha a través de FxAlta
morbilidad.FxAlta = morbilidad.FxAlta.astype('string')

In [19]:
morbilidad['fecha_al'] = pd.to_datetime({'year': morbilidad.FxAlta.str[-4:],
                           'month': morbilidad.FxAlta.str[-6:-4],
                           'day': morbilidad.FxAlta.str[:-6]
                           })

In [20]:
#creamos fecha ingreso restando a fecha_al el número de día ingresado (DiasEstancia)
morbilidad['fecha_ing'] = morbilidad.apply(lambda fila: fila['fecha_al'] - datetime.timedelta(days = fila['DiasEstancia']), axis= 1)

In [21]:
morbilidad['year']=morbilidad.fecha_ing.dt.year
morbilidad['month']=morbilidad.fecha_ing.dt.month
morbilidad['day']=morbilidad.fecha_ing.dt.day

In [22]:
morbilidad.drop(['FxAlta'],axis=1,inplace = True)

In [23]:
# Cambiamos algunos datos categóricos que vienen como números 

In [24]:
morbilidad.sample(3)

,ProvHosp,Sexo,ProvResi,DiagEntr,DiagPrin,MotivoAlta,EdadAnios,DiasEstancia,Factor,code,cap,fecha_al,fecha_ing,year,month,day
701251,29,2,29,1,F0390,1,92,14,1.00000,F03,F00-F09,2019-07-12,2019-06-28,2019,6,28
694413,8,2,8,2,F1020,1,29,5,1.02752,F10,F10-F19,2017-09-02,2017-08-28,2017,8,28
695858,28,2,28,1,F1020,1,35,2,1.00000,F10,F10-F19,2017-10-26,2017-10-24,2017,10,24


In [25]:
#diccionarios códigos
dic_sexo = {1:'Hombre',2:'Mujer'}
dic_entrada = {1:'Ordinario',2:'Urgente'}
dic_alta = {1: "Curación",2:'Traslado',3:'Fallecimiento',4:'Otras'}
num_str = {'1':'01',  '2':'02',  '3':'03',  '4':'04',  '5':'05',  '6':'06',  '7':'07',  '8':'08',  '9':'09'}

In [26]:
morbilidad.ProvHosp = morbilidad.ProvHosp.astype('string').replace(num_str)
morbilidad.ProvResi = morbilidad.ProvResi.astype('string').replace(num_str)

In [27]:
morbilidad['Entrada'] = morbilidad.DiagEntr.map(dic_entrada)
morbilidad['Alta'] = morbilidad.MotivoAlta.map(dic_alta)
morbilidad['SexNom'] = morbilidad.Sexo.map(dic_sexo)

In [28]:
#preparar provincias para geojsaon (nombre)

In [29]:
#diccionario
geo_json = r"../data/socioecon/spain_provinces.geojson"
with open (geo_json) as geo_file:
    provincias = json.load(geo_file)

cod_prov = {}
for i in range(52):    
    cod_prov[provincias['features'][i]['properties']['cod_prov']] = provincias['features'][i]['properties']['name']

In [30]:
morbilidad['ProvHosp_nom'] = morbilidad.ProvHosp.map(cod_prov)
morbilidad['ProvResi_nom'] = morbilidad.ProvResi.map(cod_prov)

In [31]:
#prov resi 53 = extranjeros
morbilidad.ProvResi_nom.fillna('Extranjeros',inplace = True)

In [32]:
morbilidad.isna().sum()

ProvHosp        0
Sexo            0
ProvResi        0
DiagEntr        0
DiagPrin        0
MotivoAlta      0
EdadAnios       0
DiasEstancia    0
Factor          0
code            0
cap             0
fecha_al        0
fecha_ing       0
year            0
month           0
day             0
Entrada         0
Alta            0
SexNom          0
ProvHosp_nom    0
ProvResi_nom    0
dtype: int64

In [33]:
morbilidad.columns

Index(['ProvHosp', 'Sexo', 'ProvResi', 'DiagEntr', 'DiagPrin', 'MotivoAlta',
       'EdadAnios', 'DiasEstancia', 'Factor', 'code', 'cap', 'fecha_al',
       'fecha_ing', 'year', 'month', 'day', 'Entrada', 'Alta', 'SexNom',
       'ProvHosp_nom', 'ProvResi_nom'],
      dtype='object')

In [34]:
morbilidad.rename({'EdadAnios':'Edad','DiasEstancia':'Dias'},axis=1,inplace = True)

In [35]:
morbilidad.sample(3)

,ProvHosp,Sexo,ProvResi,DiagEntr,DiagPrin,MotivoAlta,Edad,Dias,Factor,code,...,fecha_al,fecha_ing,year,month,day,Entrada,Alta,SexNom,ProvHosp_nom,ProvResi_nom
753175,08,2,08,1,F5000,1,17,327,1.02915,F50,...,2016-03-09,2015-04-17,2015,4,17,Ordinario,Curación,Mujer,Barcelona,Barcelona
753868,29,1,11,1,F259,1,32,67,1.00000,F25,...,2019-09-13,2019-07-08,2019,7,8,Ordinario,Curación,Hombre,Málaga,Cádiz
750031,08,2,08,1,F4325,2,16,50,1.02915,F43,...,2016-08-24,2016-07-05,2016,7,5,Ordinario,Traslado,Mujer,Barcelona,Barcelona


In [36]:
morbilidad.to_csv('../data/morbilidad_Fgen.csv',index=False)

In [63]:
dicc_poly = {pf['properties']['name']:pf['geometry'] for pf in provincias['features']}

In [64]:
morbilidad['geometry'] = morbilidad.ProvHosp_nom.map(dicc_poly)

In [65]:
morbilidad.sample(3)

,ProvHosp,Sexo,ProvResi,DiagEntr,DiagPrin,MotivoAlta,Edad,Dias,Factor,code,...,fecha_ing,year,month,day,Entrada,Alta,SexNom,ProvHosp_nom,ProvResi_nom,geometry
742884,46,1,46,2,F3110,4,42,18,1.00000,F31,...,2018-10-27,2018,10,27,Urgente,Otras,Hombre,Valencia,Valencia,"{'type': 'MultiPolygon', 'coordinates': [[[[-0..."
722785,28,1,28,2,F1420,4,50,38,1.02096,F14,...,2019-10-15,2019,10,15,Urgente,Otras,Hombre,Madrid,Madrid,"{'type': 'MultiPolygon', 'coordinates': [[[[-3..."
767044,52,1,52,2,F918,1,25,7,1.13292,F91,...,2016-02-24,2016,2,24,Urgente,Curación,Hombre,Melilla,Melilla,"{'type': 'MultiPolygon', 'coordinates': [[[[-2..."


In [ ]:
def data2geojson(df):
    features = []
    insert_features = lambda X: features.append(
            geojson.Feature(geometry=geojson.Point((X["long"],
                                                    X["lat"],
                                                    X["elev"])),
                            properties=dict(name=X["name"],
                                            description=X["description"])))
    df.apply(insert_features, axis=1)
    with open('map1.geojson', 'w', encoding='utf8') as fp:
        geojson.dump(geojson.FeatureCollection(features), fp, sort_keys=True, ensure_ascii=False)